In [123]:
from keras.datasets import mnist
import keras
import tensorflow as tf
import numpy as np
import seaborn as sns
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = train_X/255.0
test_X = test_X/255.0

PART 1

In [124]:
#without regularization
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(units=128, activation='sigmoid'),
    tf.keras.layers.Dense(units=10)
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

weights_noReg = []

for i in range(10):
    losses = model.fit(train_X, train_y, epochs=1)
    currentWeights = 0
    for j in model.layers[1].get_weights()[0]:
        for k in j:
            currentWeights += abs(k)
    for j in model.layers[2].get_weights()[0]:
        for k in j:
            currentWeights += abs(k)
    weights_noReg.append(currentWeights)


1875/1875 [==============================] - 9s 5ms/step - loss: 0.0344


In [125]:
#with regularization
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(units=128, activation='sigmoid', kernel_regularizer='l2'),
    tf.keras.layers.Dense(units=10, kernel_regularizer='l2')
])

model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

weights_Reg = []

for i in range(10):
    losses = model.fit(train_X, train_y, epochs=1)
    currentWeights = 0
    for j in model.layers[1].get_weights()[0]:
        for k in j:
            currentWeights += abs(k)
    for j in model.layers[2].get_weights()[0]:
        for k in j:
            currentWeights += abs(k)
    weights_Reg.append(currentWeights)

1875/1875 [==============================] - 12s 6ms/step - loss: 1.3716


In [126]:
import plotly.graph_objects as go

In [127]:
fig = go.Figure()
fig.add_trace(go.Bar(x=[i for i in range(1,11)], y=weights_noReg, name='Without Regularization'))
fig.add_trace(go.Bar(x=[i for i in range(1,11)], y=weights_Reg, name='With Regularization'))
fig.update_layout(xaxis_title='Epoch', yaxis_title='Magnitude of Weights', title='Verified Regularization')

The overall magnitude of the weights of the regularized model should be lower. This is because the cost function is updated to penalize larger weights.

PART 2

In [128]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [129]:
#put images in an array
images = {'Good':[],
          'Small':[],
          'Large':[]}
for i in range(3):
    for j in range(10):
        img = load_img(rf'C:\Users\andre\Documents\GitHub\AIRepository\solutions\Lab3Data\{["Good", "Small", "Large"][i]}{j}.png')
        img = img_to_array(img)
        img=img/255.0
        img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
        img = 1-img
        images[['Good', 'Small', 'Large'][i]].append(img)

In [130]:
goodPredictions = model.predict(np.array(images['Good']))
goodPredictions = [list(i).index(max(i)) for i in goodPredictions]

total = 0
for i in range(10):
    if goodPredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} good images')

1/1 [==============================] - 0s 202ms/step
Correctly Predicted 3 good images


In [131]:
smallPredictions = model.predict(np.array(images['Small']))
smallPredictions = [list(i).index(max(i)) for i in smallPredictions]

total = 0
for i in range(10):
    if smallPredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} small images')

1/1 [==============================] - 0s 45ms/step
Correctly Predicted 3 small images


In [132]:
largePredictions = model.predict(np.array(images['Large']))
largePredictions = [list(i).index(max(i)) for i in largePredictions]

total = 0
for i in range(10):
    if largePredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} large images')

1/1 [==============================] - 0s 60ms/step
Correctly Predicted 1 large images


In [133]:
print(f'Good Results: {goodPredictions}')
print(f'Small Results: {smallPredictions}')
print(f'Large Results: {largePredictions}')

Good Results: [0, 0, 6, 3, 4, 8, 4, 1, 3, 6]
Small Results: [5, 1, 9, 6, 7, 5, 5, 7, 9, 5]
Large Results: [7, 2, 7, 2, 7, 7, 2, 7, 5, 4]


PART 3

In [134]:
#without extra data
smallmodel = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(units=128, activation='sigmoid'),
    tf.keras.layers.Dense(units=10)
])

smallmodel.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

losses = smallmodel.fit(train_X, train_y, epochs=10)


Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3988
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1929
Epoch 3/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.1406
Epoch 4/10
1875/1875 [==============================] - 10s 6ms/step - loss: 0.1085
Epoch 5/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0866
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0711
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0584
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0495
Epoch 9/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.0414
Epoch 10/10
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0346


In [135]:
train_X_Left = []
for i in train_X:
    train_X_Left.append(np.roll(i, 1, 0))

train_X_Right = []
for i in train_X:
    train_X_Right.append(np.roll(i, -1, 0))

train_X_Up = []
for i in train_X:
    train_X_Up.append(np.roll(i, 1, 1))

train_X_Down = []
for i in train_X:
    train_X_Down.append(np.roll(i, -1, 1))

train_X_Left = np.array(train_X_Left)
train_X_Right = np.array(train_X_Right)
train_X_Up = np.array(train_X_Up)
train_X_Down = np.array(train_X_Down)

train_X = np.concatenate((train_X, train_X_Left, train_X_Right, train_X_Up, train_X_Down))
train_y = np.array(list(train_y) * 5)

In [136]:
#with extra data
largemodel = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(units=128, activation='sigmoid'),
    tf.keras.layers.Dense(units=10)
])

largemodel.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

losses = largemodel.fit(train_X, train_y, epochs=10)


Epoch 1/10
9375/9375 [==============================] - 52s 5ms/step - loss: 0.2470
Epoch 2/10
9375/9375 [==============================] - 50s 5ms/step - loss: 0.0885
Epoch 3/10
9375/9375 [==============================] - 47s 5ms/step - loss: 0.0597
Epoch 4/10
9375/9375 [==============================] - 43s 5ms/step - loss: 0.0459
Epoch 5/10
9375/9375 [==============================] - 50s 5ms/step - loss: 0.0373
Epoch 6/10
9375/9375 [==============================] - 50s 5ms/step - loss: 0.0311
Epoch 7/10
9375/9375 [==============================] - 52s 6ms/step - loss: 0.0268
Epoch 8/10
9375/9375 [==============================] - 45s 5ms/step - loss: 0.0233
Epoch 9/10
9375/9375 [==============================] - 45s 5ms/step - loss: 0.0203
Epoch 10/10
9375/9375 [==============================] - 49s 5ms/step - loss: 0.0179


In [137]:
goodPredictions = largemodel.predict(np.array(images['Good']))
goodPredictions = [list(i).index(max(i)) for i in goodPredictions]

total = 0
for i in range(10):
    if goodPredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} good images')

1/1 [==============================] - 0s 270ms/step
Correctly Predicted 8 good images


In [138]:
smallPredictions = largemodel.predict(np.array(images['Small']))
smallPredictions = [list(i).index(max(i)) for i in smallPredictions]

total = 0
for i in range(10):
    if smallPredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} small images')

1/1 [==============================] - 0s 90ms/step
Correctly Predicted 3 small images


In [139]:
largePredictions = largemodel.predict(np.array(images['Large']))
largePredictions = [list(i).index(max(i)) for i in largePredictions]

total = 0
for i in range(10):
    if largePredictions[i] == i:
        total += 1

print(f'Correctly Predicted {total} large images')

1/1 [==============================] - 0s 60ms/step
Correctly Predicted 0 large images


In [140]:
print(f'Good Results: {goodPredictions}')
print(f'Small Results: {smallPredictions}')
print(f'Large Results: {largePredictions}')

Good Results: [0, 5, 2, 3, 4, 5, 6, 7, 8, 7]
Small Results: [7, 4, 2, 6, 9, 9, 4, 7, 9, 9]
Large Results: [2, 7, 3, 2, 7, 6, 2, 6, 5, 6]


Below will be metrics comparing the small and large data models

In [146]:
smallPredictions = smallmodel.predict(test_X)
smallPredictions = [list(i).index(max(i)) for i in smallPredictions]
largePredictions = largemodel.predict(test_X)
largePredictions = [list(i).index(max(i)) for i in largePredictions]


313/313 [==============================] - 1s 4ms/step


In [148]:
def accuracy(predicted, true):
    correctPredictions = 0
    for i in range(len(predicted)):
        if predicted[i] == true[i]:
            correctPredictions += 1
    return correctPredictions/len(predicted)

In [153]:
def oneClassPrecision(predicted, true, cat):
    truePositives = 0
    falsePositives = 0
    for i in range(len(predicted)):
        if predicted[i] == cat:
            if true[i] == cat:
                truePositives += 1
            else:
                falsePositives += 1
    return truePositives / (truePositives + falsePositives)

In [154]:
def precision(predicted, true):
    classPrecisions = []
    for i in range(0, 10):
        classPrecisions.append(oneClassPrecision(predicted, true, i))
    return np.mean(classPrecisions)

In [156]:
def oneClassRecall(predicted, true, cat):
    truePositives = 0
    falseNegatives = 0
    for i in range(len(predicted)):
        if true[i] == cat:
            if predicted[i] == cat:
                truePositives += 1
            else:
                falseNegatives += 1
    return truePositives / (truePositives + falseNegatives)

In [157]:
def recall(predicted, true):
    classRecalls = []
    for i in range(0, 10):
        classRecalls.append(oneClassRecall(predicted, true, i))
    return np.mean(classRecalls)

In [160]:
print('*****SMALL DATA*****')
print(f'Accuracy: {accuracy(smallPredictions, test_y)}')
print(f'Precision: {precision(smallPredictions, test_y)}')
print(f'Recall: {recall(smallPredictions, test_y)}')
print()
print('*****LARGE DATA*****')
print(f'Accuracy: {accuracy(largePredictions, test_y)}')
print(f'Precision: {precision(largePredictions, test_y)}')
print(f'Recall: {recall(largePredictions, test_y)}')


*****SMALL DATA*****
Accuracy: 0.9781
Precision: 0.9781591806664869
Recall: 0.9777091434215033

*****LARGE DATA*****
Accuracy: 0.9851
Precision: 0.9850155815409145
Recall: 0.9849699536460574
